In [1]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, edge_preserving_smoothing_3d, image_smoothing_gaussian_3d
from skimage.morphology import remove_small_objects  
from tifffile import imsave

import pandas as pd

In [31]:
# Myosin DataFrame

manifest_csv = pd.read_csv('manifest.csv')
myosin_only = manifest_csv[(manifest_csv['structure_name'] == 'MYH10')]
myosin_only.head(n=6)

,FOVId,raw_fov_path,PlateId,InstrumentId,structure_name,WellId,FPChannel,BFChannel
563,563,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0
569,569,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0
572,572,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0
576,576,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0
581,581,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0
587,587,/allen/aics/microscopy/Calysta/projects/traini...,12-Apr-21,ZSD1,MYH10,Undefined,1,0


In [32]:
myosin_only.raw_fov_path[587]

'/allen/aics/microscopy/Calysta/projects/training_emt/data/5500000249/ZSD1/alignV2/5500000429_10X_myosin_mode_A_3-alignV2-Scene-04-P23-B04.tiff'

In [2]:
filename = "//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000652_DD_1-01/aligned_split/5500000652_DD_1-01_AcquisitionBlock7_pt7_Scene-0_aligned.ome.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Attempted file (//allen/aics/microscopy/PRODUCTION/PIPELINE_8_1/5500000652_DD_1-01/aligned_split/5500000652_DD_1-01_AcquisitionBlock7_pt7_Scene-0_aligned.ome.tiff) load with reader: aicsimageio.readers.bfio_reader.OmeTiledTiffReader failed with error: No module named 'bfio'
The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 4, 40, 1200, 1800)


In [3]:
#####################
## PARAMETER ##
structure_channel = 1
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
# view(single_fluorescent_view(struct_img0))

<h3> Pre-Processing <h3>

In [4]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [0]
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with edge preserving smoothing 
structure_img_smooth = edge_preserving_smoothing_3d(struct_img)


intensity normalization: min-max normalization with NO absoluteintensity upper bound


<h3> Core Algorithm <h3>

In [5]:
################################
## PARAMETERS for this step ## [[scale_1, cutoff_1], [scale_2, cutoff_2]]
f3_param = [[1,0.005],[0.5,0.0018]] # [[2,0.2],[1,0.015]] current: [[2,0.0007],[1,0.0005]]
################################

bw = filament_3d_wrapper(structure_img_smooth, f3_param)

<h3> Post-Processing <h3>

In [6]:
################################
## PARAMETERS for this step ##
minArea = 4 # original: 4
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False) #connectivity = original: 1

In [7]:
# save the file
out=seg.astype(np.uint8)
out[out>0]=255
imsave("myh10-200ms-sc0.tiff", out)
#imsave("myh10-sc04-p23-b04.tiff", out)